<a href="https://colab.research.google.com/github/bunyamin-polat/SparkNLP-Document-Classifier-with-MTSamples-Dataset/blob/main/MTSamples_Document_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Setup

In [1]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr.json to spark_nlp_for_healthcare_spark_ocr.json


In [2]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 140 kB 20.8 MB/s 
     |████████████████████████████████| 198 kB 59.2 MB/s 
     |████████████████████████████████| 144 kB 38.7 MB/s 
     |████████████████████████████████| 95 kB 3.9 MB/s 
     |████████████████████████████████| 66 kB 6.0 MB/s 


In [3]:
import json
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.common import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import warnings
warnings.filterwarnings('ignore')

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())


spark = sparknlp_jsl.start(license_keys['SECRET'])

spark

Spark NLP Version : 3.4.0
Spark NLP_JSL Version : 3.4.0


In [4]:
import pandas as pd
import numpy as np

from pyspark.sql import functions as F

from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer

from pyspark.ml.classification import LogisticRegression

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


# Import MTSamples Data
- MTSamples data was read separately for each category and converted into a spark dataframe.

- Added category information to each dataframe

- Then all the dataframes were merged, "text" and "category" information was taken and saved as a spark dataframe.


In [ ]:
# textFiles1 = spark.sparkContext.wholeTextFiles("/content/drive/MyDrive/Colab Notebooks/SparkNLP/dataset/train-data/Gastroenterology/*.txt")
# textFiles2 = spark.sparkContext.wholeTextFiles("/content/drive/MyDrive/Colab Notebooks/SparkNLP/dataset/train-data/Neurology/*.txt")
# textFiles3 = spark.sparkContext.wholeTextFiles("/content/drive/MyDrive/Colab Notebooks/SparkNLP/dataset/train-data/Orthopedic/*.txt")
# textFiles4 = spark.sparkContext.wholeTextFiles("/content/drive/MyDrive/Colab Notebooks/SparkNLP/dataset/train-data/Radiology/*.txt")
# textFiles5 = spark.sparkContext.wholeTextFiles("/content/drive/MyDrive/Colab Notebooks/SparkNLP/dataset/train-data/Urology/*.txt") 

# spark_df1 = textFiles1.toDF(schema=["path","text"])
# spark_df2 = textFiles2.toDF(schema=["path","text"])
# spark_df3 = textFiles3.toDF(schema=["path","text"])
# spark_df4 = textFiles4.toDF(schema=["path","text"])
# spark_df5 = textFiles5.toDF(schema=["path","text"])

# spark_df1 = spark_df1.withColumn("category",F.lit("Gastroenterology"))
# spark_df2 = spark_df2.withColumn("category",F.lit("Neurology"))
# spark_df3 = spark_df3.withColumn("category",F.lit("Orthopedic"))
# spark_df4 = spark_df4.withColumn("category",F.lit("Radiology"))
# spark_df5 = spark_df5.withColumn("category",F.lit("Urology"))

In [ ]:
# spark_df = spark_df1.union(spark_df2.union(spark_df3.union(spark_df4.union(spark_df5))))

# spark_df = spark_df.select("text","category")


## Data Preprocessing and Save

- Since there are blank lines and html tags in our data, the related cleaning processes were performed by applying the normalization process.

- Then our data was saved to the drive in parquet format.

In [7]:
# documentAssembler = DocumentAssembler() \
#     .setInputCol('text') \
#     .setOutputCol('document')

# cleanUpPatterns = ["<[^>]*>"]

# documentNormalizer = DocumentNormalizer() \
#     .setInputCols("document") \
#     .setOutputCol("normalizedDocument") \
#     .setAction("clean") \
#     .setPatterns(cleanUpPatterns) \
#     .setReplacement("") \
#     .setPolicy("pretty_all") \
#     .setLowercase(False)

# docPatternRemoverPipeline = Pipeline() \
#     .setStages([
#         documentAssembler,
#         documentNormalizer])
    
# empty_df = spark.createDataFrame([['']]).toDF("text")

# pipelineModel = docPatternRemoverPipeline.fit(empty_df)

In [ ]:
# result_df = pipelineModel.transform(spark_df)

# result_df = result_df.select("category", F.explode('normalizedDocument.result').alias('text'))

# result_df.write.parquet("/content/drive/MyDrive/mtsamples.parquet")

# Read MTSamples Data

In [12]:
spark_df = spark.read.parquet("/content/drive/MyDrive/SparkNLPTask/mtsamples.parquet")

spark_df.show(truncate=100)

+----------+----------------------------------------------------------------------------------------------------+
|  category|                                                                                                text|
+----------+----------------------------------------------------------------------------------------------------+
|Orthopedic| PREOPERATIVE DIAGNOSES 1. EMG-proven left carpal tunnel syndrome. 2. Tenosynovitis of the left t...|
|Orthopedic| PREOPERATIVE DIAGNOSIS: Bunion, left foot. POSTOPERATIVE DIAGNOSIS: Bunion, left foot. PROCEDURE...|
|Orthopedic| RICE stands for the most important elements of treatment for many injuries---rest, ice, compress...|
|Orthopedic| The patient is an 84-year-old retired male who is referred to our office by Dr. O. He comes in t...|
|Orthopedic| PREOPERATIVE DIAGNOSES: 1. Left carpal tunnel syndrome (354.0). 2. Left ulnar nerve entrapment a...|
|Orthopedic| PREOPERATIVE DIAGNOSIS: Herniated nucleus pulposus T8-T9. POSTOPERATIVE DIA

In [13]:
spark_df = spark_df.filter(spark_df.text != "")
# None values in the text column of our data were removed from the data

In [16]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  110|
|       Neurology|  141|
|      Orthopedic|  222|
|Gastroenterology|  157|
|       Radiology|  188|
+----------------+-----+



In [8]:
!wget https://raw.githubusercontent.com/kavgan/clinical-concepts/master/clinical-stopwords.txt

# Since we will build a model with clinical data, clinical stop words data was used instead of general stopwords data.

--2022-01-24 01:06:42--  https://raw.githubusercontent.com/kavgan/clinical-concepts/master/clinical-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6582 (6.4K) [text/plain]
Saving to: ‘clinical-stopwords.txt’

clinical-stopwords. 100%[===================>]   6.43K  --.-KB/s    in 0s      

2022-01-24 01:06:42 (69.6 MB/s) - ‘clinical-stopwords.txt’ saved [6582/6582]



In [9]:
with open ('clinical-stopwords.txt', 'r') as f:
    stops = f.readlines()

# Model with Count Vectorizer + LogReg

In [10]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text")\
    .setOutputCol("document")\

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setCleanupPatterns(["[^\w\d\s]"])

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setStopWords(stops)\
    .setCaseSensitive(False)

lemmatizer = LemmatizerModel.pretrained("lemma_antbnc") \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

countVectors = CountVectorizer(inputCol="token_features", outputCol="features", vocabSize=10000, minDF=5)

label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")

nlpPipeline = Pipeline(stages=[
            documentAssembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            lemmatizer,
            finisher,
            countVectors,
            label_stringIdx
            ])


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [17]:
nlp_model = nlpPipeline.fit(spark_df)

processed_cv = nlp_model.transform(spark_df)

processed_cv.count()

818

In [18]:
processed_cv.select("features","label","category").show(truncate = 80)

+--------------------------------------------------------------------------------+-----+----------+
|                                                                        features|label|  category|
+--------------------------------------------------------------------------------+-----+----------+
|(4848,[0,1,2,3,4,6,7,8,12,13,15,16,17,20,21,25,27,28,32,33,35,36,45,47,48,50,...|  0.0|Orthopedic|
|(4848,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,25,27,28,29,30,31...|  0.0|Orthopedic|
|(4848,[0,1,2,3,4,5,6,7,8,10,12,13,16,20,23,24,25,27,28,29,30,31,32,34,37,43,4...|  0.0|Orthopedic|
|(4848,[0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,17,18,19,20,23,24,25,26,28,29,30,31...|  0.0|Orthopedic|
|(4848,[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,19,20,22,23,25,26,27,28,29,30...|  0.0|Orthopedic|
|(4848,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,18,19,20,21,23,25,27,28,29,30,32...|  0.0|Orthopedic|
|(4848,[0,1,2,3,4,5,6,7,8,9,16,19,22,24,29,31,37,42,43,49,52,59,61,63,70,77,80...|  0.0|Orthopedic|


In [19]:
(trainingData, testData) = processed_cv.randomSplit([0.8, 0.2], seed = 42)

trainingData.count()

679

In [20]:
testData.count()

139

In [21]:
trainingData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   84|
|       Neurology|  124|
|      Orthopedic|  192|
|Gastroenterology|  125|
|       Radiology|  154|
+----------------+-----+



In [22]:
testData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   26|
|       Neurology|   17|
|      Orthopedic|   30|
|Gastroenterology|   32|
|       Radiology|   34|
+----------------+-----+



In [23]:
lr = LogisticRegression(maxIter = 10, regParam=0.3)

lrModel = lr.fit(trainingData)

predictions_cv = lrModel.transform(testData)

predictions_cv_train = lrModel.transform(trainingData)

# To check the overfit-underfit status of our model, an estimate was also taken from the train data.

In [24]:
y_true = predictions_cv.select("label")
y_true = y_true.toPandas()

y_pred = predictions_cv.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.76      0.73      0.75        30
         1.0       0.70      0.82      0.76        34
         2.0       0.97      0.88      0.92        32
         3.0       0.50      0.47      0.48        17
         4.0       1.00      0.96      0.98        26

    accuracy                           0.80       139
   macro avg       0.78      0.77      0.78       139
weighted avg       0.81      0.80      0.80       139

0.7985611510791367


In [25]:
predictions_cv.select("label","category").distinct().show()

+-----+----------------+
|label|        category|
+-----+----------------+
|  4.0|         Urology|
|  0.0|      Orthopedic|
|  1.0|       Radiology|
|  3.0|       Neurology|
|  2.0|Gastroenterology|
+-----+----------------+



In [26]:
preds_df = predictions_cv.select("label").toPandas()

preds = pd.DataFrame(confusion_matrix(list(y_true.label.astype(int)), list(y_pred.prediction.astype(int))), columns = np.unique(preds_df['label']), index =  np.unique(preds_df['label']))

preds

,0.0,1.0,2.0,3.0,4.0
0.0,22,5,0,3,0
1.0,2,28,0,4,0
2.0,0,3,28,1,0
3.0,5,4,0,8,0
4.0,0,0,1,0,25


In [27]:
y_true = predictions_cv_train.select("label")
y_true = y_true.toPandas()

y_pred = predictions_cv_train.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.98      0.93      0.96       192
         1.0       0.77      0.86      0.81       154
         2.0       0.99      0.93      0.96       125
         3.0       0.83      0.84      0.84       124
         4.0       1.00      0.98      0.99        84

    accuracy                           0.90       679
   macro avg       0.92      0.91      0.91       679
weighted avg       0.91      0.90      0.91       679

0.9042709867452136


# Model with TF-IDF + LogReg

In [28]:
hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures", numFeatures=10000)

idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

In [29]:
# Updated pipeline for TF-IDF
nlpPipeline_tf = Pipeline(stages=[
            documentAssembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            lemmatizer,
            finisher,
            hashingTF,
            idf,
            label_stringIdx
            ])

In [30]:
nlp_model_tf = nlpPipeline_tf.fit(spark_df)

processed_tf = nlp_model_tf.transform(spark_df)

In [31]:
processed_tf.select("features","label","category").show(truncate = 80)

+--------------------------------------------------------------------------------+-----+----------+
|                                                                        features|label|  category|
+--------------------------------------------------------------------------------+-----+----------+
|(10000,[86,95,138,157,164,186,228,264,269,278,286,298,325,330,333,454,486,488...|  0.0|Orthopedic|
|(10000,[47,63,86,120,130,138,157,164,174,178,195,255,258,266,284,298,303,422,...|  0.0|Orthopedic|
|(10000,[7,57,78,86,222,316,354,374,387,415,427,488,515,527,563,690,739,790,79...|  0.0|Orthopedic|
|(10000,[47,49,63,79,130,144,157,164,174,199,209,237,253,260,279,286,291,298,2...|  0.0|Orthopedic|
|(10000,[95,122,130,131,157,165,266,278,286,328,330,343,358,368,453,488,494,50...|  0.0|Orthopedic|
|(10000,[47,157,171,222,255,279,303,330,418,453,488,493,494,528,592,674,690,76...|  0.0|Orthopedic|
|(10000,[1,95,164,255,263,264,303,309,343,361,488,546,563,624,648,690,804,940,...|  0.0|Orthopedic|


In [32]:
(trainingData, testData) = processed_tf.randomSplit([0.8, 0.2], seed = 42)

lrModel_tf = lr.fit(trainingData)

predictions_tf = lrModel_tf.transform(testData)

predictions_tf_train = lrModel_tf.transform(trainingData)

In [33]:
y_true = predictions_tf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_tf.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.76      0.73      0.75        30
         1.0       0.68      0.82      0.75        34
         2.0       0.97      0.91      0.94        32
         3.0       0.60      0.53      0.56        17
         4.0       1.00      0.92      0.96        26

    accuracy                           0.81       139
   macro avg       0.80      0.78      0.79       139
weighted avg       0.81      0.81      0.81       139

0.8057553956834532


In [34]:
preds = pd.DataFrame(confusion_matrix(list(y_true.label.astype(int)), list(y_pred.prediction.astype(int))), columns = np.unique(preds_df['label']), index =  np.unique(preds_df['label']))

preds

,0.0,1.0,2.0,3.0,4.0
0.0,22,6,0,2,0
1.0,2,28,0,4,0
2.0,0,3,29,0,0
3.0,4,4,0,9,0
4.0,1,0,1,0,24


In [35]:
y_true = predictions_tf_train.select("label")
y_true = y_true.toPandas()

y_pred = predictions_tf_train.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       192
         1.0       0.80      0.79      0.80       154
         2.0       0.97      0.95      0.96       125
         3.0       0.82      0.86      0.84       124
         4.0       0.99      0.99      0.99        84

    accuracy                           0.90       679
   macro avg       0.91      0.91      0.91       679
weighted avg       0.91      0.90      0.90       679

0.9042709867452136


# Model with Clinical Embeddings + LogReg

In [36]:
clinical_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "lemma"])\
    .setOutputCol("embeddings")\
    .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_sentence_embeddings"]) \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")

nlp_pipeline_w2v = Pipeline(
    stages=[documentAssembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            lemmatizer,
            clinical_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            explodeVectors,
            label_stringIdx])

nlp_model_w2v = nlp_pipeline_w2v.fit(spark_df)

processed_w2v = nlp_model_w2v.transform(spark_df)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [37]:
(trainingData, testData) = processed_w2v.randomSplit([0.8, 0.2], seed = 42)

In [38]:
lr = LogisticRegression(maxIter=10, regParam=0.3)

lrModel_w2v = lr.fit(trainingData)

predictions_w2v = lrModel_w2v.transform(testData)

predictions_w2v_train = lrModel_w2v.transform(trainingData)

In [39]:
y_true = predictions_w2v.select("label")
y_true = y_true.toPandas()

y_pred = predictions_w2v.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.82      0.94      0.87        33
         1.0       0.90      0.47      0.62        38
         2.0       0.75      0.83      0.79        29
         3.0       0.46      0.65      0.54        17
         4.0       0.64      0.73      0.68        22

    accuracy                           0.72       139
   macro avg       0.71      0.72      0.70       139
weighted avg       0.75      0.72      0.71       139

0.7194244604316546


In [40]:
preds = pd.DataFrame(confusion_matrix(list(y_true.label.astype(int)), list(y_pred.prediction.astype(int))), columns = np.unique(preds_df['label']), index =  np.unique(preds_df['label']))

preds

,0.0,1.0,2.0,3.0,4.0
0.0,31,0,0,2,0
1.0,5,18,3,11,1
2.0,0,0,24,0,5
3.0,1,2,0,11,3
4.0,1,0,5,0,16


In [41]:
y_true = predictions_w2v_train.select("label")
y_true = y_true.toPandas()

y_pred = predictions_w2v_train.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.82      0.91      0.86       189
         1.0       0.77      0.50      0.60       150
         2.0       0.83      0.95      0.89       128
         3.0       0.61      0.71      0.66       124
         4.0       0.89      0.83      0.86        88

    accuracy                           0.78       679
   macro avg       0.78      0.78      0.77       679
weighted avg       0.78      0.78      0.77       679

0.7790868924889544


# Model with Clinical Embeddings + ClassifierDL

In [42]:
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setBatchSize(8)\
  .setMaxEpochs(200)\
  .setLr(0.003)\
  .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemmatizer, 
            clinical_embeddings,
            embeddingsSentence,
            classsifierdl])


In [43]:
%%time

clf_pipelineModel = clf_pipeline.fit(trainingData)

preds_clf = clf_pipelineModel.transform(testData)

CPU times: user 1.56 s, sys: 203 ms, total: 1.77 s
Wall time: 4min 3s


In [44]:
!cd ~/annotator_logs/ && ls -lt

total 16
-rw-r--r-- 1 root root 14274 Jan 24 01:30 ClassifierDLApproach_7ce988c38242.log


In [45]:
!cat ~/annotator_logs/ClassifierDLApproach_7ce988c38242.log

Training started - epochs: 200 - learning_rate: 0.003 - batch_size: 8 - training_examples: 679 - classes: 5
Epoch 0/200 - 1.06s - loss: 133.55841 - acc: 0.269983 - batches: 85
Epoch 1/200 - 0.64s - loss: 127.5862 - acc: 0.40391153 - batches: 85
Epoch 2/200 - 0.66s - loss: 121.575554 - acc: 0.46938774 - batches: 85
Epoch 3/200 - 0.65s - loss: 117.88572 - acc: 0.5616496 - batches: 85
Epoch 4/200 - 0.71s - loss: 115.18629 - acc: 0.6048044 - batches: 85
Epoch 5/200 - 0.63s - loss: 113.69715 - acc: 0.61075675 - batches: 85
Epoch 6/200 - 0.56s - loss: 112.78897 - acc: 0.61819726 - batches: 85
Epoch 7/200 - 0.62s - loss: 112.134186 - acc: 0.61968535 - batches: 85
Epoch 8/200 - 0.60s - loss: 111.62992 - acc: 0.61968535 - batches: 85
Epoch 9/200 - 0.63s - loss: 111.22677 - acc: 0.61968535 - batches: 85
Epoch 10/200 - 0.58s - loss: 110.89424 - acc: 0.61968535 - batches: 85
Epoch 11/200 - 0.60s - loss: 110.62036 - acc: 0.62117344 - batches: 85
Epoch 12/200 - 0.59s - loss: 110.3809 - acc: 0.624149

In [46]:
preds_df = preds_clf.select("category","text","class.result").toPandas()

preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.59      0.83      0.69        29
       Neurology       0.32      0.35      0.33        17
      Orthopedic       0.73      0.91      0.81        33
       Radiology       0.63      0.63      0.63        38
         Urology       0.00      0.00      0.00        22

        accuracy                           0.60       139
       macro avg       0.45      0.54      0.49       139
    weighted avg       0.51      0.60      0.55       139



# ClassifierDL with BioBert Embeddings

In [14]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

In [ ]:
bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased", "en", "public/models")\
    .setInputCols(["document", "lemma"])\
    .setOutputCol("embeddings")\

embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setBatchSize(8)\
    .setMaxEpochs(64)\
    .setLr(0.003)\
    .setEnableOutputLogs(True)

biobert_clf_pipeline = Pipeline(
    stages = [
        documentAssembler,
        tokenizer,
        normalizer,
        stopwords_cleaner,
        lemmatizer,
        bert_embeddings,
        embeddingsSentence,
        embeddings_finisher,
        explodeVectors,
        classifierdl
    ])

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]


In [ ]:
%%time 

biobert_clf_model = biobert_clf_pipeline.fit(trainingData)

CPU times: user 3.16 s, sys: 344 ms, total: 3.51 s
Wall time: 8min 38s


In [ ]:
!cd ~/annotator_logs/ && ls -lt

total 24
-rw-r--r-- 1 root root  4565 Jan 21 21:58 ClassifierDLApproach_2c3e44ceb058.log
-rw-r--r-- 1 root root 14276 Jan 21 21:37 ClassifierDLApproach_e9fd9d2c5aba.log


In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_2c3e44ceb058.log

Training started - epochs: 64 - learning_rate: 0.003 - batch_size: 8 - training_examples: 679 - classes: 5
Epoch 0/64 - 0.75s - loss: 136.97824 - acc: 0.23384354 - batches: 85
Epoch 1/64 - 0.52s - loss: 136.95721 - acc: 0.28592688 - batches: 85
Epoch 2/64 - 0.50s - loss: 136.95636 - acc: 0.28592688 - batches: 85
Epoch 3/64 - 0.52s - loss: 137.02136 - acc: 0.28592688 - batches: 85
Epoch 4/64 - 0.50s - loss: 137.02878 - acc: 0.28592688 - batches: 85
Epoch 5/64 - 0.51s - loss: 137.02463 - acc: 0.28592688 - batches: 85
Epoch 6/64 - 0.51s - loss: 137.0191 - acc: 0.28592688 - batches: 85
Epoch 7/64 - 0.51s - loss: 137.01468 - acc: 0.28592688 - batches: 85
Epoch 8/64 - 0.51s - loss: 137.01083 - acc: 0.28592688 - batches: 85
Epoch 9/64 - 0.52s - loss: 137.00761 - acc: 0.28592688 - batches: 85
Epoch 10/64 - 0.50s - loss: 137.00467 - acc: 0.28592688 - batches: 85
Epoch 11/64 - 0.52s - loss: 137.00291 - acc: 0.28592688 - batches: 85
Epoch 12/64 - 0.51s - loss: 137.00082 - acc: 0.28592688 - batche

In [ ]:
preds = biobert_clf_model.transform(testData)

preds_df = preds.select("category","text","class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.00      0.00      0.00        32
       Neurology       0.00      0.00      0.00        17
      Orthopedic       0.22      1.00      0.36        30
       Radiology       0.00      0.00      0.00        34
         Urology       0.00      0.00      0.00        26

        accuracy                           0.22       139
       macro avg       0.04      0.20      0.07       139
    weighted avg       0.05      0.22      0.08       139



# ClassifierDL with Universal Sentence Embeddings


In [47]:
use = UniversalSentenceEncoder.pretrained()\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setBatchSize(8)\
    .setMaxEpochs(300)\
    .setLr(0.003)\
    .setEnableOutputLogs(True)\

use_clf_pipeline = Pipeline(
    stages = [
        documentAssembler,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [50]:
%%time
use_clf_pipelineModel = use_clf_pipeline.fit(trainingData)

CPU times: user 1.56 s, sys: 180 ms, total: 1.74 s
Wall time: 4min 48s


In [51]:
!cd ~/annotator_logs/ && ls -lt

total 40
-rw-r--r-- 1 root root 21089 Jan 24 01:38 ClassifierDLApproach_4f8255b59147.log
-rw-r--r-- 1 root root 14274 Jan 24 01:30 ClassifierDLApproach_7ce988c38242.log


In [52]:
!cat ~/annotator_logs/ClassifierDLApproach_4f8255b59147.log

Training started - epochs: 300 - learning_rate: 0.003 - batch_size: 8 - training_examples: 679 - classes: 5
Epoch 0/300 - 0.77s - loss: 128.26047 - acc: 0.41602892 - batches: 85
Epoch 1/300 - 0.58s - loss: 108.36165 - acc: 0.66156465 - batches: 85
Epoch 2/300 - 0.56s - loss: 103.60581 - acc: 0.68239796 - batches: 85
Epoch 3/300 - 0.61s - loss: 101.861786 - acc: 0.698767 - batches: 85
Epoch 4/300 - 0.57s - loss: 99.64581 - acc: 0.72257656 - batches: 85
Epoch 5/300 - 0.60s - loss: 97.907196 - acc: 0.7389456 - batches: 85
Epoch 6/300 - 0.56s - loss: 96.58372 - acc: 0.74489796 - batches: 85
Epoch 7/300 - 0.57s - loss: 95.618904 - acc: 0.7493623 - batches: 85
Epoch 8/300 - 0.58s - loss: 94.85293 - acc: 0.761267 - batches: 85
Epoch 9/300 - 0.56s - loss: 94.236275 - acc: 0.76870745 - batches: 85
Epoch 10/300 - 0.58s - loss: 93.73015 - acc: 0.77019554 - batches: 85
Epoch 11/300 - 0.55s - loss: 93.307365 - acc: 0.77168363 - batches: 85
Epoch 12/300 - 0.58s - loss: 92.94341 - acc: 0.7821003 - ba

In [53]:
preds = use_clf_pipelineModel.transform(testData)

preds_df = preds.select("category","text","class.result").toPandas()

preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.73      0.83      0.77        29
       Neurology       0.42      0.65      0.51        17
      Orthopedic       0.87      0.82      0.84        33
       Radiology       0.79      0.61      0.69        38
         Urology       0.85      0.77      0.81        22

        accuracy                           0.73       139
       macro avg       0.73      0.73      0.73       139
    weighted avg       0.76      0.73      0.74       139



In [54]:
use_clf_pipelineModel.stages

[DocumentAssembler_6811ed9f8d15,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_dac17b73dbe1]

In [56]:
use_clf_pipelineModel.stages[2].write().overwrite().save("/content/drive/MyDrive/SparkNLPTask/use_clf_e300_b8")

# Prediction Pipeline

In [57]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained()\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

loaded_clf_model = ClassifierDLModel.load("/content/drive/MyDrive/SparkNLPTask/use_clf_e300_b8")\

prediction_pipeline = Pipeline(stages = [
      documentAssembler,
      use,
      loaded_clf_model,
  ])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = prediction_pipeline.fit(empty_data)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [62]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [63]:
text1 = """
PREOPERATIVE DIAGNOSIS: Anemia. PROCEDURE: Upper gastrointestinal endoscopy. POSTOPERATIVE DIAGNOSES: 1. Severe duodenitis. 2. Gastroesophageal junction small ulceration seen. 3. No major bleeding seen in the stomach. PROCEDURE IN DETAIL: The patient was put in left lateral position. Olympus scope was inserted from the mouth, under direct visualization advanced to the upper part of the stomach, upper part of esophagus, middle of esophagus, GE junction, and some intermittent bleeding was seen at the GE junction. Advanced into the upper part of the stomach into the antrum. The duodenum showed extreme duodenitis and the scope was then brought back. Retroflexion was performed, which was normal. Scope was then brought back slowly. Duodenitis was seen and a little bit of ulceration seen at GE junction. FINDING: Severe duodenitis, may be some source of bleeding from there, but no active bleeding at this time.
"""

result = light_model.annotate(text1)

result['class']

['Gastroenterology']

In [64]:
text1 = """
PREOPERATIVE DIAGNOSES: 1. Epidural hematoma, cervical spine. 2. Status post cervical laminectomy, C3 through C7 postop day #10. 3. Central cord syndrome. 4. Acute quadriplegia. POSTOPERATIVE DIAGNOSES: 1. Epidural hematoma, cervical spine. 2. Status post cervical laminectomy, C3 through C7 postop day #10. 3. Central cord syndrome. 4. Acute quadriplegia. PROCEDURE PERFORMED: 1. Evacuation of epidural hematoma. 2. Insertion of epidural drain. ANESTHESIA: General. COMPLICATIONS: None. ESTIMATED BLOOD LOSS: 200 cc. HISTORY: This is a 64-year-old female who has had an extensive medical history beginning with coronary artery bypass done on emergent basis while she was in Maryland in April of 2003 after having myocardial infarction. She was then transferred to Beaumont Hospital, at which point, she developed a sternal abscess. The patient was treated for the abscess in Beaumont and then subsequently transferred to some other type of facility near her home in Warren, Michigan at which point, she developed a second what was termed minor myocardial infarction. The patient subsequently recovered in a Cardiac Rehab Facility and approximately two weeks later, brings us to the month of August, at which time she was at home ambulating with a walker or a cane, and then sustained a fall and at that point she was unable to walk and had acute progressive weakness and was identified as having a central cord syndrome based on an MRI, which showed record signal change. The patient underwent cervical laminectomy and seemed to be improving subjectively in terms of neurologic recovery, but objectively there was not much improvement. Approximately 10 days after the surgery, brings us to today's date, the health officer was notified of the patient's labored breathing. When she examined the patient, she also noted that the patient was unable to move her extremities. She was concerned and called the Orthopedic resident who identified the patient to be truly quadriplegic. I was notified and ordered the operative crew to report immediately and recommended emergent decompression for the possibility of an epidural hematoma. On clinical examination, there was swelling in the posterior aspect of the neck. The patient has no active movement in the upper and lower extremity muscle groups. Reflexes are absent in the upper and lower extremities. Long track signs are absent. Sensory level is at the C4 dermatome. Rectal tone is absent. I discussed the findings with the patient and also the daughter. We discussed the possibility of this is permanent quadriplegia, but at this time, the compression of the epidural space was warranted and certainly for exploration reasons be sure that there is a hematoma there and they have agreed to proceed with surgery. They are aware that it is possible she had known permanent neurologic status regardless of my intervention and they have agreed to accept this and has signed the consent form for surgery
"""

result = light_model.annotate(text1)

result['class']

['Orthopedic']

In [17]:
# test = spark.sparkContext.wholeTextFiles("/content/drive/MyDrive/SparkNLPTask/dataset/test-data/*.txt")

# spark_test = test.toDF(schema=["path","text"])

# spark_test = spark_test.select("text")


In [16]:
# test = pipelineModel.transform(spark_test)

# test = test.select(F.explode('normalizedDocument.result').alias('text'))

# test.write.parquet("/content/drive/MyDrive/SparkNLPTask/mtsamples_test.parquet")

In [ ]:
test = spark.read.parquet("/content/drive/MyDrive/SparkNLPTask/mtsamples_test.parquet")

test = test.filter(test.text != "")

test.show(truncate=100)

In [61]:
pd.set_option("display.max_colwidth", 3000)

preds = prediction_model.transform(test)

result_df = preds.select("text","class.result").toPandas()

result_df.head(30)

,text,result
0,"PREOPERATIVE DIAGNOSIS: Anemia. PROCEDURE: Upper gastrointestinal endoscopy. POSTOPERATIVE DIAGNOSES: 1. Severe duodenitis. 2. Gastroesophageal junction small ulceration seen. 3. No major bleeding seen in the stomach. PROCEDURE IN DETAIL: The patient was put in left lateral position. Olympus scope was inserted from the mouth, under direct visualization advanced to the upper part of the stomach, upper part of esophagus, middle of esophagus, GE junction, and some intermittent bleeding was seen at the GE junction. Advanced into the upper part of the stomach into the antrum. The duodenum showed extreme duodenitis and the scope was then brought back. Retroflexion was performed, which was normal. Scope was then brought back slowly. Duodenitis was seen and a little bit of ulceration seen at GE junction. FINDING: Severe duodenitis, may be some source of bleeding from there, but no active bleeding at this time.",[Gastroenterology]
1,"CC: Lethargy. HX: This 28y/o RHM was admitted to a local hospital on 7/14/95 for marked lethargy. He had been complaining of intermittent headaches and was noted to have subtle changes in personality for two weeks prior to 7/14/95. On the morning of 7/14/95, his partner found him markedly lethargic and complaingin of abdominal pain and vomiting. He denied fevers, chills, sweats, cough, CP, SOB or diarrhea. Upon evaluation locally, he had a temperature of 99.5F and appeared lethargic. He also had anisocoria with left pupil 0.5mm bigger than the right. There was also question of left facial weakness. An MRI was obtained and revealed a large left hemispheric mass lesion with surrounding edema and mass effect. He was given 10mg of IV Decardron,100gm of IV Mannitol, intubated and hyperventilated and transferred to UIHC. He was admitted to the Department of Medicine on 7/14/95, and transferred to the Department of Neurology on 7/17/95, after being extubated. MEDS ON ADMISSION: Bactrim DS qd, Diflucan 100mg qd, Acyclovir 400mg bid, Xanax, Stavudine 40mg bid, Rifabutin 300mg qd. PMH: 1) surgical correction of pyoloric stenosis, age 1, 2)appendectomy, 3) HIV/AIDS dx 1991. He was initially treated with AZT, then DDI. He developed chronic diarrhea and was switched to D4T in 1/95. However, he developed severe neuropathy and this was stopped 4/95. The diarrhea recured. He has Acyclovir resistant genital herpes and generalized psoriasis. He most recent CD4 count (within 1 month of admission) was 20. FHX: HTN and multiple malignancies of unknown type. SHX: Homosexual, in monogamous relationship with an HIV infected partner for the past 3 years. EXAM: 7/14/95 (by Internal Medicine): BP134/80, HR118, RR16 on vent, 38.2C, Intubated. MS: Somnolent, but opened eyes to loud voices and would follow most commands. CN: Pupils 2.5/3.0 and &quot;equally reactive to light.&quot; Mild horizontal nystagmus on rightward gaze. EOM were otherwise intact. MOTOR: Moved 4 extremities well. Sensory/Coord/Gait/Station/Reflexes: not done. Gen EXAM: Penil ulcerations. EXAM: 7/17/96 (by Neurology): BP144/73, HR59, RR20, 36.0, extubated. MS: Alert and mildly lethargic. Oriented to name only. Thought he was a local hospital and that it was 1/17/1994. Did not understand he had a brain lesion. CN: Pupils 6/5.5 decreasing to 4/4 on exposure to light. EOM were full and smooth. No RAPD or light-near dissociation. papilledema (OU). Right lower facial weakness and intact facial sensation to PP testing. Gag-shrug and corneal responses were intact, bilaterally. Tongue midline. MOTOR: Grade 5- strength on the right side. Sensory: no loss of sensation on PP/VIB/PROP testing. Coord: reduced speed and accuracy on right FNF and right HKS movements. Station: RUE pronator drift. Gait: not done. Reflexes: 2+/2 throughout. Babinski sign present on right and absent on left. Gen Exam: unremarkable except for the genital lesion noted by Internal medicine. COURSE: The outside MRI was reviewed and was not...",[Neurology]
2,"EXAM: Barium enema. CLINICAL H